In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 60kB/s 
     |████████████████████████████████| 204kB 44.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=e91ce8193fdb5cbb7393bac0e2196ccb7cac0caf81649d9d8507474d3e21e4cd
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 43 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Organización de Datos 75.06. Segundo Cuatrimestre de 2018. Examen parcial, segunda oportunidad

1) Spotify cuenta con un log de todas las canciones que fueron escuchadas en su plataforma, esta información se encuentra en un RDD que está paralelizado por día, es decir cada día es una partición. Los campos del RDD son los siguientes: (date, user_id, song_id, song_title, artist).  
Se cuenta por otro lado con un RDD que asigna "tags" a las canciones, por ejemplo "rock, punk, actual, top-10, acoustic, etc). Una canción puede tener asociados "n" tags. El RDD tiene el formato (song_id, tag).  
Cada día se corre un proceso para asignar el tag "rising" a las canciones que se escucharon mas veces el día de hoy que el día de ayer. Estos nuevos tags pasan a formar parte del RDD de tags para futuros usos. Programar en PySpark usando el API de RDD lo solicitado. 
 (15 pts) (***) 

In [ ]:
canciones_escuchadas = [('2019-01-20', 1000 , 1000 , 'Give life back to music', 'Daft punk'),
             ('2019-01-20', 1001 , 1000 , 'Give life back to music', 'Daft punk'),
             ('2019-01-21', 1002 , 1000 , 'Give life back to music', 'Daft punk'),
             ('2019-01-21', 1003 , 1000 , 'Give life back to music', 'Daft punk'),
             ('2019-01-21', 1004 , 1001 , 'Music', 'Madonna'),
             ('2019-01-22', 1005 , 1001 , 'Music', 'Madonna'),
             ('2019-01-22', 1001 , 1001 , 'Music', 'Madonna'),
             ('2019-01-21', 1002 , 1005 , 'Dany California', 'RHCP'),
             ('2019-01-21', 1003 , 1005 , 'Dany California', 'RHCP'),
             ('2019-01-21', 1004 , 1005 , 'Dany California', 'RHCP'),
             ('2019-01-22', 1005 , 1005 , 'Dany California', 'RHCP'),
             ('2019-01-21', 1001 , 1009 , 'Enter Sandman', 'Metallica'),
             ('2019-01-22', 1002 , 1009 , 'Enter Sandman', 'Metallica'),
             ('2019-01-21', 1003 , 1010 , 'Me against the music', 'Britney Spears'),
              ('2019-01-22', 1004 , 1010 , 'Me against the music', 'Britney Spears'),
              ('2019-01-22', 1005 , 1010 , 'Me against the music', 'Britney Spears'),
              ('2019-01-22', 1001 , 1010 , 'Me against the music', 'Britney Spears')]

In [ ]:
canciones = [(1000,'electronic, pop, modern, hit, futuristic, trendy'),
             (1001, 'hit, trendy, pop, classic'),
             (1005, 'hit, funk, rock, la, california'),
             (1009, 'rock, heavy, gutural, hit, racing, classic'),
             (1010, 'vintage, hit, pop, blondie, sexy')]

In [ ]:
canciones_escuchadasRDD = sc.parallelize(canciones_escuchadas)
cancionesRDD = sc.parallelize(canciones)

In [ ]:
canciones_escuchadasRDD.collect()

[('2019-01-20', 1000, 1000, 'Give life back to music', 'Daft punk'),
 ('2019-01-20', 1001, 1000, 'Give life back to music', 'Daft punk'),
 ('2019-01-21', 1002, 1000, 'Give life back to music', 'Daft punk'),
 ('2019-01-21', 1003, 1000, 'Give life back to music', 'Daft punk'),
 ('2019-01-21', 1004, 1001, 'Music', 'Madonna'),
 ('2019-01-22', 1005, 1001, 'Music', 'Madonna'),
 ('2019-01-22', 1001, 1001, 'Music', 'Madonna'),
 ('2019-01-21', 1002, 1005, 'Dany California', 'RHCP'),
 ('2019-01-21', 1003, 1005, 'Dany California', 'RHCP'),
 ('2019-01-21', 1004, 1005, 'Dany California', 'RHCP'),
 ('2019-01-22', 1005, 1005, 'Dany California', 'RHCP'),
 ('2019-01-21', 1001, 1009, 'Enter Sandman', 'Metallica'),
 ('2019-01-22', 1002, 1009, 'Enter Sandman', 'Metallica'),
 ('2019-01-21', 1003, 1010, 'Me against the music', 'Britney Spears'),
 ('2019-01-22', 1004, 1010, 'Me against the music', 'Britney Spears'),
 ('2019-01-22', 1005, 1010, 'Me against the music', 'Britney Spears'),
 ('2019-01-22', 1001, 

In [ ]:
cancionesRDD.collect()

[(1000, 'electronic, pop, modern, hit, futuristic, trendy'),
 (1001, 'hit, trendy, pop, classic'),
 (1005, 'hit, funk, rock, la, california'),
 (1009, 'rock, heavy, gutural, hit, racing, classic'),
 (1010, 'vintage, hit, pop, blondie, sexy')]

In [ ]:
escuchadas_aux = canciones_escuchadasRDD.map(lambda x: ((x[2],x[0]),1))
escuchadas_aux.collect()

[((1000, '2019-01-20'), 1),
 ((1000, '2019-01-20'), 1),
 ((1000, '2019-01-21'), 1),
 ((1000, '2019-01-21'), 1),
 ((1001, '2019-01-21'), 1),
 ((1001, '2019-01-22'), 1),
 ((1001, '2019-01-22'), 1),
 ((1005, '2019-01-21'), 1),
 ((1005, '2019-01-21'), 1),
 ((1005, '2019-01-21'), 1),
 ((1005, '2019-01-22'), 1),
 ((1009, '2019-01-21'), 1),
 ((1009, '2019-01-22'), 1),
 ((1010, '2019-01-21'), 1),
 ((1010, '2019-01-22'), 1),
 ((1010, '2019-01-22'), 1),
 ((1010, '2019-01-22'), 1)]

In [ ]:
cantidad_veces_escuchadas = escuchadas_aux.reduceByKey(lambda x,y: (x+y) )
cantidad_veces_escuchadas.collect()

[((1000, '2019-01-20'), 2),
 ((1000, '2019-01-21'), 2),
 ((1010, '2019-01-21'), 1),
 ((1010, '2019-01-22'), 3),
 ((1001, '2019-01-21'), 1),
 ((1001, '2019-01-22'), 2),
 ((1005, '2019-01-21'), 3),
 ((1005, '2019-01-22'), 1),
 ((1009, '2019-01-21'), 1),
 ((1009, '2019-01-22'), 1)]

In [ ]:
cantidad_veces_escuchadas = (cantidad_veces_escuchadas.map(lambda x: (x[0][0], x[1]) ))
cantidad_veces_escuchadas.collect()

[(1000, 2),
 (1000, 2),
 (1010, 1),
 (1010, 3),
 (1001, 1),
 (1001, 2),
 (1005, 3),
 (1005, 1),
 (1009, 1),
 (1009, 1)]

In [ ]:
cantidad_veces_escuchadas_comparada = (cantidad_veces_escuchadas.reduceByKey(lambda x,y:[x,y]))
cantidad_veces_escuchadas_comparada.collect()

[(1000, [2, 2]),
 (1010, [1, 3]),
 (1001, [1, 2]),
 (1005, [3, 1]),
 (1009, [1, 1])]

In [ ]:
cantidad_veces_escuchadas_comparada = cantidad_veces_escuchadas_comparada.filter(lambda x: x[1][0]<x[1][1])
cantidad_veces_escuchadas_comparada.collect()

[(1010, [1, 3]), (1001, [1, 2])]

In [ ]:
rising_tags = cantidad_veces_escuchadas_comparada.join(cancionesRDD)
rising_tags.collect()

[(1001, ([1, 2], 'hit, trendy, pop, classic')),
 (1010, ([1, 3], 'vintage, hit, pop, blondie, sexy'))]

In [ ]:
rising_tags_considerar = rising_tags.map(lambda x: (x[1][1]))
rising_tags_considerar.collect()

['hit, trendy, pop, classic', 'vintage, hit, pop, blondie, sexy']

#FINAL